In [3]:
import os
from PIL import Image

import math

import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import LeakyReLU, Input
from keras.layers import BatchNormalization
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD
from keras import metrics
from keras.regularizers import l2

import keras.backend as K

In [4]:
train_dir = r'D:\pythonProject\hansik\final_food\Training'
val_dir = r'D:\pythonProject\hansik\final_food\Validation'
test_dir = r'D:\pythonProject\hansik\final_food\Test'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale = 1./255) # 검증, 테스트 데이터는 augmentation 하면 안됨!
test_datagen = ImageDataGenerator(rescale= 1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (299, 299),
    batch_size = 32, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
    seed = 42
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (299, 299),
    batch_size = 32, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
    shuffle=True,
    seed = 42
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (299, 299),
    batch_size = 32, # gpu 연산 -> 2의 거듭제곱꼴로 맞추는 것이 좋다고 함
    seed = 42
)

Found 118486 images belonging to 150 classes.
Found 14756 images belonging to 150 classes.
Found 14928 images belonging to 150 classes.


In [6]:
from datetime import datetime

today = datetime.today()      # 현재 일 가져오기

date ='{}.{}.{}.{}h'.format(today.year, today.month, today.day, today.hour)
date

MODEL_SAVE_FOLDER_PATH = './model/{}/'.format(date)

# 디렉토리가 없는 경우에만 디렉토리 생성
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.makedirs(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '결과 : {epoch:02d}-{val_acc:.4f}.hdf5'
print(model_path)


./model/2024.5.31.14h/결과 : {epoch:02d}-{val_acc:.4f}.hdf5


In [7]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))

x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
x = Dropout(.4)(x)
predictions = Dense(150,kernel_initializer='glorot_uniform', 
                    kernel_regularizer=l2(.0005), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

opt = SGD(learning_rate=.01, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10) # 조기종료 콜백함수 정의
cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                            verbose=1, save_best_only=True) # 체크포인트 저장

In [6]:
# lr_scheduler : epoch 증가함에 따라 learning_rate를 작게 해 cosf_function이 최적값 주변에서 맴도는 것을 완화!

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)

In [8]:
history2 = model.fit(train_generator,
                    epochs=30, steps_per_epoch= len(train_generator), 
                    validation_data = val_generator,
                    validation_steps= len(val_generator),
                    callbacks=[cb_checkpoint, early_stopping, lr_scheduler]
                )

Epoch 1/30
   4/3703 [..............................] - ETA: 19:43:57 - loss: 5.6073 - accuracy: 0.0000e+00